In [1]:
import numpy as np
from ncon import ncon
import pandas as pd

In [2]:
def Random_MPS(N,d,D) :
    """""
    MPS : tensor array
    N : number of tensors in the MPS
    d : physical dimension
    D : Bond dimension
    """""
    MPS = []
    MPS.append(np.random.rand(d,D))
    for l in range(1,N-1):
        MPS.append(np.random.rand(D,d,D))
    MPS.append(np.random.rand(D,d))
    return MPS

In [3]:
import numpy as np
from ncon import ncon

def Left_Orthogonal_Form(MPS,d,D) :
    """""
    Inputs :
        MPS : tensor array
        d : physical dimension
        D : bond dimension
    Outputs :
        Unitary_MPS : the MPS tensor array composed of unitaries U
        R : the right environnment
    """""
    Unitary_MPS = []
    N = len(MPS)
    i = 0
    while i < N :
        if i == 0 :
            #SVD on the left-most (rank-2) tensor
            U,Sm,V = np.linalg.svd(MPS[i],full_matrices=True)
            S = np.zeros((MPS[i].shape)) #Building the singular values into a tensor S
            for j in range(min(MPS[i].shape)):
                S[j, j] = Sm[j]
            
            Unitary_MPS.append(U)
            i = i + 1
        
        if i == N-1 :
            #SVD on the resulting right-most (rank-2) tensor
            B_0 = ncon([S,V,MPS[i]],[[-1,1],[1,2],[2,-2]])
            U,Sm,V = np.linalg.svd(B_0,full_matrices=True)
            S = np.zeros((MPS[i].shape)) #Building the singular values into a tensor S
            for j in range(min(MPS[i].shape)):
                S[j, j] = Sm[j]

            Unitary_MPS.append(U)
            R = S
            i = i + 1
        
        else :
            #SVD on each of the (rank-3) MPS tensor
            B_0 = ncon([S,V,MPS[i]],[[-1,1],[1,2],[2,-2,-3]])
            B_1 = B_0.reshape(B_0.shape[0]*B_0.shape[1],B_0.shape[2])
            U,Sm,V = np.linalg.svd(B_1,full_matrices=True)
            
            #Truncating S, when the bond dimensions reaches D to avoid infinite growth
            if U.shape[1] > D :
                U = U.reshape(D,d,D,d)
                S = np.zeros((D,D))
                for j in range(D):
                    S[j, j] = Sm[j]       
            else :
                U = U.reshape(B_0.shape[0],B_0.shape[1],B_0.shape[0]*B_0.shape[1])
                S = np.zeros((U.shape[2],D))
                for j in range(U.shape[2]):
                    S[j, j] = Sm[j]
            
            Unitary_MPS.append(U)
            i = i + 1
            
    return Unitary_MPS,R